In [23]:
import pandas as pd
import matplotlib.pyplot as plt

In [24]:
datos_electricos = pd.read_csv("data_proyecto.csv", encoding= "utf-8")
datos_electricos.head()

,device_id,device_name,measurement_time(UTC),current(A),voltage(V),power(W),power_factor,energy(Wh)
0,82608,Linea 2 Tablero Principal,1/1/2019 0:00,0.71,135.3,93.1,0.97,23.3
1,73170,L3 Transportador Botellas PDC III,1/1/2019 0:00,2.94,127.0,355.1,0.95,88.8
2,72819,Tablero C Alumbrado Linea Produccion,1/1/2019 0:00,6.34,127.0,764.4,0.95,191.1
3,72823,L4 Control de Inyectores,1/1/2019 0:00,30.97,127.0,3736.0,0.95,934.0
4,82649,L4 Mezcladora Paramix,1/1/2019 0:00,4.79,127.0,578.0,0.95,144.5


In [26]:
datos_grupos = pd.read_excel("Device_Groups.xlsx")
datos_grupos.head()

,device_name,DG1,DG2,Usable
0,Alumbrado General L1 y Tratamiento Aguas,Potencia Industrial,Otros,0
1,Compresor 1 Aire 250 HP,Potencia Industrial,Compresores de aire,1
2,Compresor 2 Aire 200 HP,Potencia Industrial,Compresores de aire,1
3,Compresor de Aire 1 60 HP,Potencia Industrial,Compresores de aire,1
4,Compresor de Aire 2 60 HP,Potencia Industrial,Compresores de aire,1


In [27]:
datos_electricos2 = datos_electricos.drop(columns=["device_id","current(A)","voltage(V)","power_factor","energy(Wh)"])
datos_electricos2.head()

,device_name,measurement_time(UTC),power(W)
0,Linea 2 Tablero Principal,1/1/2019 0:00,93.1
1,L3 Transportador Botellas PDC III,1/1/2019 0:00,355.1
2,Tablero C Alumbrado Linea Produccion,1/1/2019 0:00,764.4
3,L4 Control de Inyectores,1/1/2019 0:00,3736.0
4,L4 Mezcladora Paramix,1/1/2019 0:00,578.0


In [28]:
datos_electricos3=pd.merge(left = datos_electricos2, right = datos_grupos, on="device_name", how = "left").drop(columns="DG1")
datos_electricos3.head()

,device_name,measurement_time(UTC),power(W),DG2,Usable
0,Linea 2 Tablero Principal,1/1/2019 0:00,93.1,Linea 2 Total,1.0
1,L3 Transportador Botellas PDC III,1/1/2019 0:00,355.1,Linea 3,0.0
2,Tablero C Alumbrado Linea Produccion,1/1/2019 0:00,764.4,Otros,0.0
3,L4 Control de Inyectores,1/1/2019 0:00,3736.0,Linea 4 Total,1.0
4,L4 Mezcladora Paramix,1/1/2019 0:00,578.0,NaN,NaN


In [32]:
datos_electricos_final = datos_electricos3[(datos_electricos3["Usable"]==1 )].drop(columns= "Usable")
datos_electricos_final.head()

,device_name,measurement_time(UTC),power(W),DG2
0,Linea 2 Tablero Principal,1/1/2019 0:00,93.1,Linea 2 Total
3,L4 Control de Inyectores,1/1/2019 0:00,3736.0,Linea 4 Total
15,L4 PDC II,1/1/2019 0:00,4251.3,Linea 4 Total
16,Compresor de Amoniaco 2 Tornillo 250 HP VFD,1/1/2019 0:00,401.3,Refrigeración L1L3L4
17,Compresor de Aire 1 60 HP,1/1/2019 0:00,7.2,Compresores de aire


In [35]:
datos_electricos_final["DG2"].unique()

array(['Linea 2 Total', 'Linea 4 Total', 'Refrigeración L1L3L4',
       'Compresores de aire', 'Linea 1 Total', 'Linea 3 Total',
       'Refrigeración L2'], dtype=object)